# Session 3: Classical Posterior Sampling (Non-ML)

### Problem Setup: Bayesian Inverse Problems

In the Bayesian framework, we treat the unknown parameter $u$ as a random variable. Our goal is to update our belief about $u$ after observing noisy data $y$.

$$y = G(u) + \eta$$

#### Bayesian Formulation
* **Prior $\rho(u)$:** Initial knowledge about $u$.
* **Likelihood $\mathbb{P}(y|u)$:** Probability of observing $y$ given $u$. Usually $\mathbb{P}(y|u) \propto \exp(-\frac{1}{2}\|y - G(u)\|^2_{\Sigma})$.
* **Posterior $\pi^y(u)$:** The distribution of $u$ given $y$. By Bayes' Theorem:

$$\pi^y(u) \propto \mathbb{P}(y|u)\rho(u)$$

Since the normalization constant (evidence) is often intractable, we use sampling to characterize the posterior.

---

### Classical Sampling Methods

<table style="width: 100%; font-size: 1.2em; border-collapse: collapse;">
  <thead>
    <tr style="background-color: #f2f2f2; border-bottom: 2px solid black;">
      <th style="padding: 12px; text-align: left;">Method</th>
      <th style="padding: 12px; text-align: left;">Strategy</th>
      <th style="padding: 12px; text-align: left;">Pros</th>
      <th style="padding: 12px; text-align: left;">Cons</th>
    </tr>
  </thead>
  <tbody>
    <tr style="border-bottom: 1px solid #ddd;">
      <td style="padding: 12px;"><strong>Importance Sampling</strong></td>
      <td style="padding: 12px;">Reweights samples from a proposal.</td>
      <td style="padding: 12px;">Parallelizable; no burn-in.</td>
      <td style="padding: 12px;">Fails in high dimensions.</td>
    </tr>
    <tr style="border-bottom: 1px solid #ddd;">
      <td style="padding: 12px;"><strong>MCMC </strong></td>
      <td style="padding: 12px;">Constructs a Markov Chain.</td>
      <td style="padding: 12px;">Exact convergence guarantees.</td>
      <td style="padding: 12px;">Correlated samples; slow convergence.</td>
    </tr>
    <tr>
      <td style="padding: 12px;"><strong>EKI</strong></td>
      <td style="padding: 12px;">Derivative-free particle update.</td>
      <td style="padding: 12px;">Fast; handles high dimensions.</td>
      <td style="padding: 12px;">Gaussian approximation bias.</td>
    </tr>
  </tbody>
</table>

## 1. Importance Sampling (IS)

Importance Sampling is a technique to estimate properties of a target distribution $\pi^y(u)$ while only having the ability to sample from a simpler proposal distribution $q(u)$.

---

### 1.1 Mathematical Foundation

Recall the posterior density:
$$\pi^y(u) = \frac{1}{Z} \ell(y|u) \rho(u)$$
where $Z = \int \ell(y|u) \rho(u) du$ is the normalization constant (evidence).

Suppose we want to compute the expectation of a function $f(u)$ under the posterior, denoted as $\mathbb{E}^{u\sim\pi^y}[f(u)]$ or $\pi^y(f)$:
$$\mathbb{E}^{u\sim\pi^y}[f(u)] = \int f(u) \pi^y(u) du = \int f(u) \frac{\frac{1}{Z} \ell(y|u) \rho(u)}{q(u)} q(u) du$$

---

### 1.2 Importance Weights

By sampling $\{u^{(i)}\}_{i=1}^N$ from the proposal $q(u)$, we can approximate the expectation using **Importance Weights**:
1.  **Unnormalized weights:** $w(u^{(i)}) = \frac{\ell(y|u^{(i)}) \rho(u^{(i)})}{q(u^{(i)})}$
2.  **Normalized weights:** $W^{(i)} = \frac{w(u^{(i)})}{\sum_{j=1}^N w(u^{(j)})}$

The empirical approximation of the expectation is then:
$$\mathbb{E}_{\pi^y}[f(u)] \approx \sum_{i=1}^N W^{(i)} f(u^{(i)})$$

---

### 1.3 Choice of Proposal Distribution $q(u)$

$q(u)$: 1. easy to sample from
2. has a support that covers the support of the posterior $\pi^y(u)$.

A straightforward choice:
$$q(u) = \rho(u)$$
**Normalized Importance weights:**
$$W^{(i)} = \frac{\ell(y|u^{(i)})}{\sum_{j=1}^N \ell(y|u^{(j)})}$$

> **Note:** While setting $q = \rho$ is simple, it can be extremely inefficient if the data $y$ is very informative (i.e., the likelihood is peaked in a region where the prior has low probability). In such cases, most weights $W^{(i)}$ will be near zero, and the estimate will be dominated by only a few particles.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, norm
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 1. Core Functions ---
def get_prior_pdf(pos):
    """
    Mixed Gaussian Prior
    pos: grid of (u1, u2) coordinates
    """
    p1 = multivariate_normal.pdf(pos, mean=[-2, -2], cov=np.eye(2))
    p2 = multivariate_normal.pdf(pos, mean=[2, 2], cov=1.5 * np.eye(2))
    return 0.5 * p1 + 0.5 * p2

def sample_prior(n):
    """
    Direct sampling from mixture prior
    n: number of samples
    """
    comp = np.random.choice([0, 1], size=n)
    samples = np.zeros((n, 2))
    samples[comp==0] = np.random.multivariate_normal([-2, -2], np.eye(2), size=np.sum(comp==0))
    samples[comp==1] = np.random.multivariate_normal([2, 2], 1.5 * np.eye(2), size=np.sum(comp==1))
    return samples

def get_likelihood_pdf(pos, y_obs, noise_std, obs_mode):
    """
    Likelihood based on H operator (mapping u to y)
    pos: grid of (u1, u2)
    """
    if obs_mode == 'Full (u1, u2)':
        return multivariate_normal.pdf(pos, mean=y_obs, cov=(noise_std**2) * np.eye(2))
    elif obs_mode == 'U1 only':
        u1_vals = pos[..., 0].flatten()
        lik = norm.pdf(u1_vals, loc=y_obs[0], scale=noise_std)
        return lik.reshape(pos.shape[:2])
    else: # 'U2 only'
        u2_vals = pos[..., 1].flatten()
        lik = norm.pdf(u2_vals, loc=y_obs[0], scale=noise_std)
        return lik.reshape(pos.shape[:2])

def get_kde_density(grid_pos, samples, weights=None, bandwidth=0.2):
    """
    Weighted Kernel Density Estimation
    """
    density = np.zeros(grid_pos.shape[:2])
    if weights is None:
        weights = np.ones(len(samples)) / len(samples)

    for i, s in enumerate(samples):
        if weights[i] > 1e-5:
            kernel = multivariate_normal.pdf(grid_pos, mean=s, cov=bandwidth**2)
            density += weights[i] * kernel
    return density

# --- 2. UI Setup ---
N_slider = widgets.IntSlider(value=100, min=20, max=1000, step=20, description='N Samples:')
y_obs_1 = widgets.FloatText(value=0.0, description='y1 (obs):')
y_obs_2 = widgets.FloatText(value=0.0, description='y2 (obs):')
obs_mode_drop = widgets.Dropdown(options=['Full (u1, u2)', 'U1 only', 'U2 only'],
                                 value='Full (u1, u2)', description='H Op:')
noise_slider = widgets.FloatSlider(value=0.5, min=0.1, max=1.5, step=0.1, description='Noise σ:')
run_button = widgets.Button(description="Run", button_style='success')
output = widgets.Output()

def run_is_analysis(b):
    with output:
        clear_output(wait=True)
        np.random.seed(42)

        N = N_slider.value
        noise_std = noise_slider.value
        obs_m = obs_mode_drop.value

        # Define y_obs based on mode
        if obs_m == 'Full (u1, u2)':
            y_obs = np.array([y_obs_1.value, y_obs_2.value])
        elif obs_m == 'U1 only':
            y_obs = np.array([y_obs_1.value])
        else:
            y_obs = np.array([y_obs_2.value])

        u1_lin = np.linspace(-6, 6, 70)
        u2_lin = np.linspace(-6, 6, 70)
        U1, U2 = np.meshgrid(u1_lin, u2_lin)
        pos = np.dstack((U1, U2))

        # 1. Analytical Components
        Z_prior = get_prior_pdf(pos)
        Z_like = get_likelihood_pdf(pos, y_obs, noise_std, obs_m)
        Z_post = Z_prior * Z_like

        # 2. Importance Sampling
        u_samples = sample_prior(N)
        if obs_m == 'Full (u1, u2)':
            u_projected = u_samples
        elif obs_m == 'U1 only':
            u_projected = u_samples[:, 0]
        else:
            u_projected = u_samples[:, 1]

        # Calculate weights: w = p(y|u)
        diff = y_obs - u_projected
        if obs_m == 'Full (u1, u2)':
            dist_sq = np.sum(diff**2, axis=-1)
        else:
            dist_sq = diff**2

        w_unnorm = np.exp(-0.5 * dist_sq / (noise_std**2))
        weights = w_unnorm / (np.sum(w_unnorm) + 1e-15)
        ess = 1.0 / (np.sum(weights**2) + 1e-15)

        # --- Plotting 2x3 ---
        fig, axes = plt.subplots(2, 3, figsize=(15, 9.5))

        # Row 1: Prior
        axes[0,0].contourf(U1, U2, Z_prior, levels=20, cmap='Greens')
        axes[0,0].set_title("1. Prior Density $\\rho(u)$")

        axes[0,1].scatter(u_samples[:, 0], u_samples[:, 1], c='seagreen', s=15, alpha=0.6)
        axes[0,1].set_title(f"2. Prior Particles (N={N})")

        Z_prior_kde = get_kde_density(pos, u_samples)
        axes[0,2].contourf(U1, U2, Z_prior_kde, levels=20, cmap='Greens')
        axes[0,2].set_title("3. Empirical Prior (KDE)")

        # Row 2: Posterior
        axes[1,0].contourf(U1, U2, Z_post, levels=20, cmap='Purples')
        axes[1,0].set_title("4. True Posterior (Unnorm)")

        # Scaling sizes for visualization
        sizes = weights * N * 25
        axes[1,1].scatter(u_samples[:, 0], u_samples[:, 1], c='indigo', s=sizes, alpha=0.6)
        axes[1,1].set_title(f"5. Weighted Particles (ESS: {ess:.1f})")

        Z_post_kde = get_kde_density(pos, u_samples, weights=weights)
        axes[1,2].contourf(U1, U2, Z_post_kde, levels=20, cmap='Purples')
        axes[1,2].set_title("6. Empirical Posterior (KDE)")

        for ax in axes.flatten():
            ax.set_aspect('equal')
            ax.set_xlim(-6, 6); ax.set_ylim(-6, 6)
            ax.set_xlabel("$u_1$"); ax.set_ylabel("$u_2$")

        plt.tight_layout()
        plt.show()

run_button.on_click(run_is_analysis)
display(widgets.VBox([
    N_slider,
    widgets.HBox([y_obs_1, y_obs_2, noise_slider, obs_mode_drop]),
    run_button
]), output)

### 1.4 Importance Sampling (IS): Further Comments

Importance Sampling serves as a critical baseline in Bayesian computation, but it faces significant challenges in practical inverse problems.

**Advantages**
* **Simplicity:** It only requires sampling from the prior $\rho(u)$ and evaluating the likelihood $\ell(y|u)$.
* **Parallelization:** Every particle is independent, making the algorithm "embarrassingly parallel" for GPU/HPC implementation.
* **Fast Results:** Unlike MCMC, there is no "burn-in" period or dependency on previous states; results are available as soon as samples are weighted.
* **Convergence Guarantee:** With a sufficiently large number of particles ($N \to \infty$), Importance Sampling can approximate the posterior distribution to any desired level of accuracy.

**Disadvantages and Challenges**
* **The Curse of Dimensionality:** In high-dimensional spaces, the "overlap" between the prior and the posterior becomes exponentially small.
* **Weight Collapse:** In most inverse problems, the likelihood is much more "peaked" than the prior. This leads to a scenario where a few particles carry almost all the weight ($W^{(i)} \approx 1$), while the rest are statistically irrelevant.
* **Effective Sample Size (ESS) Deficiency:** We quantify the quality of the IS approximation using the **Effective Sample Size**:
    $$ESS = \frac{1}{\sum_{i=1}^N (W^{(i)})^2}$$
    A low $ESS$ (relative to $N$) indicates that the variance of our estimator is extremely high, and the posterior approximation is unreliable.



> **Key Takeaway:** While IS is robust for low-dimensional or non-informative data, it becomes inefficient when the data $y$ is highly informative. This motivates the need for **MCMC**, where particles "walk" towards high-probability regions rather than waiting to be picked.

## 2. Markov Chain Monte Carlo (MCMC)

### 2.1 Motivation
In Bayesian Inverse Problems, we often seek to compute the posterior expectation $E^{x \sim \pi^y}[f(x)]$. While direct Monte Carlo methods exist, they face significant hurdles:
* **The Normalization Constant:** The posterior $\pi^y(x) = \frac{1}{Z} \rho(y-G(x))\rho(x)$ requires the constant $Z$, which is an integral often impossible to compute analytically.
* **Sampling Difficulty:** Direct sampling can be extremely difficult when the range of the likelihood $\rho(y-G(x))$ is very small or concentrated in complex regions of the state space.
* **MCMC Solution:** MCMC provides a way to deliver samples $u^{(n)} \sim \pi^y$ such that the algorithm remains independent of $Z$.

---

### 2.2 Definitions: Markov Chain & Kernel
A Markov Chain is a sequence of random variables where the next state depends only on the current state. This transition is governed by a **Markov Kernel**.

**Definition: Markov Kernel**
A Markov kernel $p: \mathbb{R}^d \times \mathbb{R}^d \rightarrow \mathbb{R}$ satisfies:
1.  $p(x, v) \ge 0$ for all $(x, v) \in \mathbb{R}^d \times \mathbb{R}^d$.
2.  $\int_{\mathbb{R}^d} p(x, v) dv = 1$ for all $x \in \mathbb{R}^d$.

In the MCMC context, we aim for the empirical distribution of these samples to approximate the posterior:
$$\pi^y \simeq \frac{1}{N} \sum_{n=1}^{N} \delta_{u^{(n)}}$$

---

### 2.3 Metropolis-Hastings (MH) Algorithm
The MH algorithm is a specific MCMC implementation that uses a "proposal and accept/reject" mechanism.

**The Algorithm:**
1.  **Initialize:** Choose an initial state $u^{(0)}$.
2.  **Iterate:** For $n \in \mathbb{Z}^{+}$:
    * **Propose:** Draw a candidate $v_n^* \sim q(x^{(n)}, \cdot)$, where $q$ is a proposal Markov kernel.
    * **Acceptance Probability:** Calculate $a(x^{(n)}, v_n^*)$:
$$a(x, v) = \min \left\{ 1, \frac{\pi^y(v) q(v, x)}{\pi^y(x) q(x, v)} \right\}$$
    * **Update:**
$$u^{(n+1)} = \begin{cases} v_n^* & \text{with probability } a(x^{(n)}, v_n^*) \\ u^{(n)} & \text{with probability } 1 - a(x^{(n)}, v_n^*) \end{cases}$$

---

### 2.4 Convergence and Rates

The validity of the Metropolis-Hastings (MH) algorithm is grounded in the theory of Markov Chain Monte Carlo, ensuring that the target posterior distribution $\pi^y$ is the unique stationary distribution of the constructed chain.

**Proposition: Detailed Balance and Stationarity**
Let $\pi^y(x)$ be the target posterior density. Let $q(x, v)$ be a proposal kernel and $a(x, v)$ be the acceptance probability defined as:
$$a(x, v) = \min \left\{ 1, \frac{\pi^y(v) q(v, x)}{\pi^y(x) q(x, v)} \right\}$$
Then the transition kernel $p(x, v) = q(x, v)a(x, v)$ (for $x \neq v$) satisfies the detailed balance condition:
$$\pi^y(x) p(x, v) = \pi^y(v) p(v, x)$$
Consequently, $\pi^y$ is a stationary distribution of the Markov chain.

**Proof:**
To show detailed balance, we consider the term $\pi^y(x) q(x, v) a(x, v)$.
Without loss of generality, assume that $\frac{\pi^y(v) q(v, x)}{\pi^y(x) q(x, v)} \leq 1$.
1. Under this assumption, $a(x, v) = \frac{\pi^y(v) q(v, x)}{\pi^y(x) q(x, v)}$.
2. Then, the left hand side becomes:
   $$\pi^y(x) q(x, v) \cdot \frac{\pi^y(v) q(v, x)}{\pi^y(x) q(x, v)} = \pi^y(v) q(v, x)$$
3. Since the ratio is $\leq 1$, its reciprocal $\frac{\pi^y(x) q(x, v)}{\pi^y(v) q(v, x)} \geq 1$, which implies $a(v, x) = 1$.
4. The right hand side becomes:
   $$\pi^y(v) q(v, x) \cdot a(v, x) = \pi^y(v) q(v, x) \cdot 1 = \pi^y(v) q(v, x)$$
Thus, $\pi^y(x) p(x, v) = \pi^y(v) p(v, x)$. Integrating both sides with respect to $x$ gives:
$$\int \pi^y(x) p(x, v) dx = \pi^y(v) \int p(v, x) dx = \pi^y(v)$$
which proves that $\pi^y$ is stationary.

---

### 2.5 Key Comments

* **Independence of $Z$:** The Metropolis-Hastings algorithm does not require knowledge of the normalization constant $Z$. This is because $Z$ cancels out in the acceptance ratio $a(x, v)$.
* **Convergence Rate:** The approximation of the posterior expectation $E^{\pi^y}[f(x)]$ typically exhibits a statistical error scaling of $\mathcal{O}(1/\sqrt{N})$.
* **Choice of Proposal $q$:**
    * **Independence Sampler:** One could choose the prior as the proposal ($q(x, v) = \rho(v)$). However, if the likelihood $\rho(y-G(x))$ is highly concentrated, most samples from the prior will fall into low-probability regions, leading to an extremely low acceptance rate.
    * **Random Walk:** A common choice is $q(x, v) = \mathcal{N}(x, \sigma^2 I)$. The step size $\sigma$ must be carefully tuned to balance the exploration of the state space (large steps) and the acceptance rate (small steps).
    * **High-dimensional Scaling:** In high-dimensional inverse problems, simple proposals often fail. Advanced methods like **pCN (preconditioned Crank-Nicolson)** are designed to be prior-invariant, ensuring the convergence rate does not collapse as the dimension increases.
* **Burn-in Period:** In practice, initial samples are often discarded to ensure the chain has "forgotten" the starting point $u^{(0)}$ and reached the stationary regime $\pi^y$.
* **Non-i.i.d. Sampling:** Unlike standard Monte Carlo, MCMC samples are correlated (non-i.i.d.). However, the Ergodic Theorem guarantees that the time average converges to the spatial average as $N \to \infty$.

---

### 2.6 Impact of the Proposal Scale ($\sigma$) in Gaussian Kernels

In a **Random Walk Metropolis** algorithm, we commonly employ a **Gaussian Transition Kernel**:
$$q(u^{(n)}, v) = \mathcal{N}(v| u^{(n)}, \sigma^2 \mathbf{\Sigma})$$
Here, $\sigma$ is the **scale parameter** (step size) that modulates the covariance $\mathbf{\Sigma}$. The choice of $\sigma$ is the primary factor determining the efficiency of the MCMC chain.

| Proposal Scale ($\sigma$) | Acceptance Rate ($a$) | Mixing Characteristics | Visual Symptom (Trace Plot) |
| :--- | :--- | :--- | :--- |
| **Too Small** | **High** ($a \to 1$) | **Inefficient Exploration:** The chain moves via a slow random walk, failing to explore the full support of $\pi^y$ in finite time. | Looks like a "smooth curve" or dense thread; high autocorrelation. |
| **Too Large** | **Low** ($a \to 0$) | **Stagnation:** Most proposals land in low-probability regions and are rejected. The chain rarely moves. | Looks like "staircases" or long flat plateaus; the chain is stuck. |
| **Optimal** | **Moderate** ($0.2 - 0.5$) | **Rapid Mixing:** Large enough steps to bridge the distribution's support, yet small enough to maintain a healthy acceptance frequency. | Looks like a "fuzzy caterpillar"; low autocorrelation and fast convergence. |



**Key Considerations for $\sigma$:**
* **Theoretical Optimum:** For a $d$-dimensional Gaussian target, the optimal acceptance rate is approximately $0.234$ as $d \to \infty$. In low dimensions ($d=1, 2$), the target rate is typically higher, around $0.44$.
* **Adaptive MCMC:** Since the optimal $\sigma$ depends on the unknown scale of the posterior, modern algorithms often "tune" $\sigma$ automatically during the burn-in period to reach a target acceptance rate.
* **Effective Sample Size (ESS):** The goal of tuning $\sigma$ is to maximize the ESS, which represents the number of independent samples equivalent to the $N$ correlated MCMC samples.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, norm
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 1. Core Functions ---
def get_prior_pdf(pos):
    """
    Mixed Gaussian Prior
    pos: grid of (u1, u2) coordinates
    """
    p1 = multivariate_normal.pdf(pos, mean=[-2, -2], cov=np.eye(2))
    p2 = multivariate_normal.pdf(pos, mean=[2, 2], cov=1.5 * np.eye(2))
    return 0.5 * p1 + 0.5 * p2

def get_likelihood_pdf(pos, y_obs, noise_std, obs_mode):
    """
    Likelihood based on different H operators
    pos: grid of (u1, u2)
    """
    if obs_mode == 'Full (u1, u2)':
        return multivariate_normal.pdf(pos, mean=y_obs, cov=(noise_std**2) * np.eye(2))
    elif obs_mode == 'U1 only':
        u1_vals = pos[..., 0].flatten()
        lik = norm.pdf(u1_vals, loc=y_obs[0], scale=noise_std)
        return lik.reshape(pos.shape[:2])
    else: # 'U2 only'
        u2_vals = pos[..., 1].flatten()
        lik = norm.pdf(u2_vals, loc=y_obs[0], scale=noise_std)
        return lik.reshape(pos.shape[:2])

def get_log_posterior(u, y_obs, noise_std, obs_mode):
    """
    Log-posterior for MCMC sampling
    u: current state [u1, u2]
    """
    prior_val = get_prior_pdf(u)

    if obs_mode == 'Full (u1, u2)':
        y_model = u
    elif obs_mode == 'U1 only':
        y_model = u[0]
    else:
        y_model = u[1]

    dist_sq = np.sum((y_obs - y_model)**2)
    log_lik = -0.5 * dist_sq / (noise_std**2)
    return np.log(prior_val + 1e-15) + log_lik

def get_kde_density(grid_pos, samples, bandwidth=0.3):
    """
    Empirical density estimation via KDE
    """
    density = np.zeros(grid_pos.shape[:2])
    sub_samples = samples[::max(1, len(samples)//400)]
    for s in sub_samples:
        kernel = multivariate_normal.pdf(grid_pos, mean=s, cov=bandwidth**2)
        density += kernel
    return density

# --- 2. UI Setup ---
n_steps_slider = widgets.IntSlider(value=2000, min=500, max=5000, description='Steps:')
burn_in_slider = widgets.FloatSlider(value=0.2, min=0.0, max=0.8, step=0.05, description='Burn-in %:')
sigma_slider = widgets.FloatSlider(value=0.5, min=0.05, max=2.0, step=0.05, description='Prop σ:')
y_obs_1 = widgets.FloatText(value=0.0, description='y1 (obs):')
y_obs_2 = widgets.FloatText(value=0.0, description='y2 (obs):')
obs_mode_drop = widgets.Dropdown(options=['Full (u1, u2)', 'U1 only', 'U2 only'],
                                 value='Full (u1, u2)', description='H Op:')
noise_slider = widgets.FloatSlider(value=1.0, min=0.1, max=2, step=0.1, description='Noise σ:')
run_button = widgets.Button(description="Run MCMC", button_style='success')
output = widgets.Output()

def run_full_analysis(b):
    with output:
        clear_output(wait=True)
        np.random.seed(42)

        N = n_steps_slider.value
        burn_idx = int(N * burn_in_slider.value)
        sigma = sigma_slider.value
        noise_std = noise_slider.value
        obs_m = obs_mode_drop.value

        # Determine y_obs vector based on mode
        if obs_m == 'Full (u1, u2)':
            y_obs = np.array([y_obs_1.value, y_obs_2.value])
        elif obs_m == 'U1 only':
            y_obs = np.array([y_obs_1.value])
        else:
            y_obs = np.array([y_obs_2.value])

        u1_lin = np.linspace(-6, 6, 80)
        u2_lin = np.linspace(-6, 6, 80)
        U1, U2 = np.meshgrid(u1_lin, u2_lin)
        pos = np.dstack((U1, U2))

        # 1. MCMC Sampling (Metropolis)
        samples = []
        curr_u = np.array([-4.0, -4.0])
        curr_lp = get_log_posterior(curr_u, y_obs, noise_std, obs_m)
        acc = 0
        for _ in range(N):
            prop_u = curr_u + sigma * np.random.randn(2)
            prop_lp = get_log_posterior(prop_u, y_obs, noise_std, obs_m)
            if np.log(np.random.rand()) < (prop_lp - curr_lp):
                curr_u, curr_lp = prop_u, prop_lp
                acc += 1
            samples.append(curr_u)
        samples = np.array(samples)
        post_samples = samples[burn_idx:]

        # 2. Plotting 2x3
        fig, axes = plt.subplots(2, 3, figsize=(15, 9))

        Z_prior = get_prior_pdf(pos)
        axes[0,0].contourf(U1, U2, Z_prior, levels=20, cmap='Greens')
        axes[0,0].set_title("1. Prior Density $\\rho(u)$")

        Z_like = get_likelihood_pdf(pos, y_obs, noise_std, obs_m)
        axes[0,1].contourf(U1, U2, Z_like, levels=20, cmap='Reds')
        axes[0,1].set_title(f"2. Likelihood ({obs_m})")

        Z_post = Z_prior * Z_like
        axes[0,2].contourf(U1, U2, Z_post, levels=20, cmap='Purples')
        axes[0,2].set_title("3. True Posterior (Unnorm)")

        # MCMC Diagnostics
        axes[1,0].contour(U1, U2, Z_post, levels=5, alpha=0.3)
        axes[1,0].plot(samples[:burn_idx, 0], samples[:burn_idx, 1], 'r-', alpha=0.3, lw=0.5, label='Burn-in')
        axes[1,0].plot(post_samples[:, 0], post_samples[:, 1], 'b-', alpha=0.4, lw=0.5, label='Samples')
        axes[1,0].set_title(f"4. MCMC Path (Acc: {acc/N:.2f})")
        axes[1,0].legend()

        axes[1,1].plot(samples[:, 0], color='teal', lw=0.7)
        axes[1,1].axvline(x=burn_idx, color='red', linestyle='--', label='Burn-in')
        axes[1,1].set_title("5. Trace Plot ($u_1$)")
        axes[1,1].set_xlabel("Iterations")

        Z_kde = get_kde_density(pos, post_samples)
        axes[1,2].contourf(U1, U2, Z_kde, levels=20, cmap='Purples')
        axes[1,2].set_title("6. MCMC Empirical Density")

        for i in range(2):
            for j in range(3):
                if not (i==1 and j==1):
                    axes[i,j].set_aspect('equal')
                    axes[i,j].set_xlim(-6, 6); axes[i,j].set_ylim(-6, 6)
                    axes[i,j].set_xlabel("$u_1$"); axes[i,j].set_ylabel("$u_2$")

        plt.tight_layout()
        plt.show()

run_button.on_click(run_full_analysis)
display(widgets.VBox([
    widgets.HBox([n_steps_slider, burn_in_slider, sigma_slider]),
    widgets.HBox([y_obs_1, y_obs_2, noise_slider, obs_mode_drop]),
    run_button
]), output)

## 3. Ensemble Kalman Inversion (EKI)

### 3.1 Motivation
In high-dimensional Bayesian Inverse Problems, MCMC methods often suffer from the "curse of dimensionality" and high computational costs per sample. **Ensemble Kalman Inversion (EKI)** provides a derivative-free, ensemble-based framework to solve inverse problems by treating optimization as a dynamical system.

* **Derivative-Free:** EKI avoids the need for the adjoint or gradient of the forward operator $G(u)$, utilizing empirical ensemble covariances instead.
* **Optimization Focus:** While inspired by the Kalman Filter, EKI is typically applied **iteratively** to minimize a data-misfit functional.
* **Prior-Based Initialization:** The algorithm is initialized by sampling $N$ particles $\{u_0^{(j)}\}_{j=1}^N$ from the **prior distribution** $\rho(u)$. This initial ensemble determines the searchable subspace for the entire inversion process.

---

### 3.2 The Optimization Objectives
There is a distinction between the **global goal** of the algorithm and the **variational target** of each individual update step.

**1. Global Goal (The Data-Misfit):**
EKI seeks to find a parameter vector $u$ that minimizes the negative log-likelihood of the observations:
$$\min_{u \in \mathbb{R}^d} \Phi(u; y) = \frac{1}{2} \|y - G(u)\|^2_{\Gamma}$$

**2. Step-wise Variational Target (Matching):**
Each iteration $n \to n+1$ for a particle $j$ is the solution to a local variational problem:
$$\min_{u} J_n^{(j)}(u) = \underbrace{\frac{1}{2} \|u - u_n^{(j)}\|_{(C_n^{uu})^{-1}}^2}_{\text{State Matching}} + \underbrace{\frac{1}{2} \|y - G(u)\|_{\Gamma^{-1}}^2}_{\text{Observation Matching}}$$
* **State Matching:** Acts as a "trust region" or proximity constraint, ensuring the particle does not jump further than the current ensemble's uncertainty ($C_n^{uu}$) allows.
* **Observation Matching:** Drives the particle to reduce the misfit between the model output and the data $y$.



---

### 3.3 The EKI Update Rule
The discrete-time iteration for EKI at step $n$ is defined by:
$$u_{n+1}^{(j)} = u_n^{(j)} + C_n^{uG} (C_n^{GG} + \Gamma)^{-1} (y - G(u_n^{(j)}))$$

**Components:**
1.  **Empirical Cross-Covariance ($C_n^{uG}$):**
$$C_n^{uG} = \frac{1}{N-1} \sum_{k=1}^N (u_n^{(k)} - \bar{u}_n) \otimes (G(u_n^{(k)}) - \bar{G}_n)$$
2.  **Empirical Model-Output Covariance ($C_n^{GG}$):**
$$C_n^{GG} = \frac{1}{N-1} \sum_{k=1}^N (G(u_n^{(k)}) - \bar{G}_n) \otimes (G(u_n^{(k)}) - \bar{G}_n)$$
where $\bar{u}_n$ and $\bar{G}_n$ are the ensemble means.

---

### 3.4 Linear Gaussian Case: Exactness and One-Step Approximation
The EKI update is theoretically exact when $G(u) = Au$ is linear and the prior is Gaussian.

**Proposition: Exactness in Linear Gaussian Setting**
Assume $G(u) = Au$. If we consider a single update step with $\Delta t = 1$ and $N \to \infty$:
1. The empirical mean $\bar{u}_1$ converges to the **true Bayesian posterior mean**.
2. The empirical covariance $C_1^{uu}$ converges to the **true Bayesian posterior covariance**.

**Proof Sketch:**
Substituting $G(u) = Au$ into the update rule:
$$u_1^{(j)} = u_0^{(j)} + C_0^{uu}A^T(AC_0^{uu}A^T + \Gamma)^{-1}(y - Au_0^{(j)})$$
As $N \to \infty$, $C_0^{uu} \to C_0$ (the prior covariance). By the Woodbury Identity, this update is algebraically identical to the mean and covariance of the Gaussian posterior $\mathcal{N}(m_{post}, C_{post})$ where $C_{post} = (C_0^{-1} + A^T \Gamma^{-1} A)^{-1}$.

---

### 3.5 Continuous-Time Limit: Gradient Flow
For non-linear problems, EKI is used as a **multi-step** solver. By introducing a time step $h$ and letting $h \to 0$, we obtain the Gradient Flow ODE:
$$\frac{du^{(j)}}{dt} = -C(u) \nabla_u \Phi(u^{(j)}; y)$$

This shows that EKI is a **Preconditioned Gradient Descent**. The ensemble covariance $C(u)$ acts as a derivative-free preconditioner (an approximation of the inverse Hessian) that adapts the descent direction based on the current ensemble spread.



---

### 3.6 Comparison of MCMC and EKI

| Feature | MCMC (Metropolis-Hastings) | EKI (Ensemble Kalman Inversion) |
| :--- | :--- | :--- |
| **Output** | Correlated samples from $\pi^y$. | Ensemble evolving toward optimizer. |
| **Linear Gaussian** | Exact as $N_{iter} \to \infty$. | **Exact in 1 step** as $N \to \infty$. |
| **Non-linear** | Theoretically exact posterior. | Biased; drives toward **Consensus**. |
| **Ensemble Size ($N$)** | $N=1$ (usually single chain). | Crucial for capturing covariance rank. |
| **Computational Cost** | High (Sequential). | Low (Highly Parallelizable). |

---

### 3.7 Key Theoretical Comments
* **Subspace Property:** All particles $\{u_n^{(j)}\}$ are strictly confined to the affine subspace $\mathcal{A}$ spanned by the **initial prior ensemble**.
  This serves as an implicit regularization; EKI cannot explore directions not present in the prior samples.
* **Ensemble Collapse (Consensus):** In the iterative regime, as $t \to \infty$, the ensemble variance $C_n^{uu} \to 0$. All particles converge to a single point (Consensus), which is the optimizer of $\Phi(u; y)$ within the subspace $\mathcal{A}$.
* **Implicit Regularization:** The use of the prior to initialize the ensemble and the "State Matching" term in the variational target provide a natural regularization that prevents over-fitting in ill-posed problems.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, norm
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.animation as animation

# --- 1. Core Functions ---
def get_prior_pdf(pos, mode='Mixed Gaussian'):
    """Analytical prior density"""
    if mode == 'Gaussian':
        return multivariate_normal.pdf(pos, mean=[0, 0], cov=2.0 * np.eye(2))
    else:
        p1 = multivariate_normal.pdf(pos, mean=[-2, -2], cov=np.eye(2))
        p2 = multivariate_normal.pdf(pos, mean=[2, 2], cov=1.5 * np.eye(2))
        return 0.5 * p1 + 0.5 * p2

def sample_prior(n, mode='Mixed Gaussian'):
    """Initial ensemble sampling"""
    if mode == 'Gaussian':
        return np.random.multivariate_normal([0, 0], 2.0 * np.eye(2), size=n)
    else:
        comp = np.random.choice([0, 1], size=n)
        samples = np.zeros((n, 2))
        samples[comp==0] = np.random.multivariate_normal([-2, -2], np.eye(2), size=np.sum(comp==0))
        samples[comp==1] = np.random.multivariate_normal([2, 2], 1.5 * np.eye(2), size=np.sum(comp==1))
        return samples

def get_likelihood_pdf(pos, y_obs, noise_std, obs_mode):
    """Analytical likelihood based on H"""
    if obs_mode == 'Full (u1, u2)':
        return multivariate_normal.pdf(pos, mean=y_obs, cov=(noise_std**2) * np.eye(2))
    elif obs_mode == 'U1 only':
        return norm.pdf(pos[..., 0], loc=y_obs[0], scale=noise_std)
    else:
        return norm.pdf(pos[..., 1], loc=y_obs[0], scale=noise_std)

def get_kde_density(grid_pos, samples, bandwidth=0.3):
    """Empirical density from ensemble"""
    density = np.zeros(grid_pos.shape[:2])
    for s in samples:
        kernel = multivariate_normal.pdf(grid_pos, mean=s, cov=bandwidth**2)
        density += kernel
    return density / len(samples)

# --- 2. UI Setup ---
n_slider = widgets.IntSlider(value=100, min=20, max=500, description='N (Ensemble):')
iter_slider = widgets.IntSlider(value=20, min=1, max=100, description='Iterations:')
dt_slider = widgets.FloatSlider(value=0.5, min=0.1, max=1.0, step=0.1, description='Step Size Δt:')
inf_slider = widgets.FloatSlider(value=1.0, min=1.0, max=1.1, step=0.01, description='Inflation:')
prior_drop = widgets.Dropdown(options=['Mixed Gaussian', 'Gaussian'], value='Mixed Gaussian', description='Prior:')
y1_text = widgets.FloatText(value=0.0, description='y1 (obs):')
y2_text = widgets.FloatText(value=0.0, description='y2 (obs):')
obs_mode_drop = widgets.Dropdown(options=['Full (u1, u2)', 'U1 only', 'U2 only'], value='Full (u1, u2)', description='H Op:')
noise_slider = widgets.FloatSlider(value=0.5, min=0.1, max=1.5, step=0.1, description='Noise σ:')
run_button = widgets.Button(description="Run EKI & Animate", button_style='success')
output = widgets.Output()

def run_eki_full(b):
    with output:
        clear_output(wait=True)
        np.random.seed(42)

        # Parameters
        N, T, dt, alpha = n_slider.value, iter_slider.value, dt_slider.value, inf_slider.value
        prior_m, obs_m, noise_std = prior_drop.value, obs_mode_drop.value, noise_slider.value
        Gamma = noise_std**2

        if obs_m == 'Full (u1, u2)':
            y_obs = np.array([y1_text.value, y2_text.value]); H = lambda u: u; Gamma_mat = Gamma * np.eye(2)
        elif obs_m == 'U1 only':
            y_obs = np.array([y1_text.value]); H = lambda u: u[:, 0:1]; Gamma_mat = np.array([[Gamma]])
        else:
            y_obs = np.array([y2_text.value]); H = lambda u: u[:, 1:2]; Gamma_mat = np.array([[Gamma]])

        # EKI Evolution History
        u = sample_prior(N, prior_m)
        history = [u.copy()]

        for _ in range(T):
            u_mean = np.mean(u, axis=0)
            u = u_mean + alpha * (u - u_mean) # Inflation
            w = H(u)
            w_mean = np.mean(w, axis=0)

            C_uw = (u - u_mean).T @ (w - w_mean) / (N - 1)
            C_ww = (w - w_mean).T @ (w - w_mean) / (N - 1)

            K = C_uw @ np.linalg.inv(C_ww + Gamma_mat)
            u = u + dt * ((y_obs - w) @ K.T)
            history.append(u.copy())

        # Plotting Setup (3 Rows)
        fig = plt.figure(figsize=(15, 14))
        gs = fig.add_gridspec(3, 3)
        u_lin = np.linspace(-6, 6, 80); U1, U2 = np.meshgrid(u_lin, u_lin); pos = np.dstack((U1, U2))

        # Row 1: Ground Truth
        ax1, ax2, ax3 = fig.add_subplot(gs[0,0]), fig.add_subplot(gs[0,1]), fig.add_subplot(gs[0,2])
        Z_prior = get_prior_pdf(pos, prior_m)
        Z_like = get_likelihood_pdf(pos, y_obs, noise_std, obs_m)
        Z_post = Z_prior * Z_like

        ax1.contourf(U1, U2, Z_prior, levels=20, cmap='Greens'); ax1.set_title("1. True Prior Density")
        ax2.contourf(U1, U2, Z_like, levels=20, cmap='Reds'); ax2.set_title(f"2. True Likelihood ({obs_m})")
        ax3.contourf(U1, U2, Z_post, levels=20, cmap='Purples'); ax3.set_title("3. True Posterior (Unnorm)")

        # Row 2: EKI Results
        ax4, ax5, ax6 = fig.add_subplot(gs[1,0]), fig.add_subplot(gs[1,1]), fig.add_subplot(gs[1,2])
        ax4.scatter(history[0][:,0], history[0][:,1], c='seagreen', s=15, alpha=0.6); ax4.set_title("4. Initial Ensemble")
        ax5.scatter(u[:,0], u[:,1], c='indigo', s=15, alpha=0.6); ax5.set_title(f"5. Final Ensemble (t={T})")
        Z_kde = get_kde_density(pos, u)
        ax6.contourf(U1, U2, Z_kde, levels=20, cmap='Purples'); ax6.set_title("6. Final Ensemble KDE")

        # Row 3: Animation Axis
        ax_anim = fig.add_subplot(gs[2, :])
        ax_anim.contour(U1, U2, Z_post, levels=8, alpha=0.2, cmap='Purples')
        scat = ax_anim.scatter(history[0][:,0], history[0][:,1], c='teal', s=30, edgecolors='k', alpha=0.7)
        ax_anim.set_title("EKI Particle Flow Animation")

        for ax in [ax1, ax2, ax3, ax4, ax5, ax6, ax_anim]:
            ax.set_aspect('equal'); ax.set_xlim(-6, 6); ax.set_ylim(-6, 6)

        def update(frame):
            scat.set_offsets(history[frame])
            ax_anim.set_xlabel(f"Iteration: {frame} / {T}")
            return scat,

        plt.tight_layout()
        ani = animation.FuncAnimation(fig, update, frames=len(history), interval=100, blit=True)
        plt.close() # Prevents static plot display
        display(widgets.HTML(ani.to_jshtml()))

run_button.on_click(run_eki_full)
display(widgets.VBox([
    widgets.HBox([n_slider, iter_slider, dt_slider, inf_slider]),
    widgets.HBox([prior_drop, y1_text, y2_text, noise_slider, obs_mode_drop]),
    run_button
]), output)